In [1]:
import sys
import os
from pprint import pprint as ppt
# sys.path.append(f"{os.path.dirname(__file__)}/../")
sys.path.append('../')
from botcore.utils import load_sample_qa, QAMemory

p, q, a = load_sample_qa()
memory = QAMemory()
memory.load_all(p, q, a)
print(memory.memory)

Load done
chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Eating peanuts made my face swollen.', additional_kwargs={}, example=False), AIMessage(content='What other foods have you eaten recently?', additional_kwargs={}, example=False), HumanMessage(content='Peanuts', additional_kwargs={}, example=False), AIMessage(content='Have you had any other symptoms besides swelling?', additional_kwargs={}, example=False), HumanMessage(content='Hives', additional_kwargs={}, example=False), AIMessage(content='How long did it take for your symptoms to start after eating the peanuts?', additional_kwargs={}, example=False), HumanMessage(content='Less than 1 hour', additional_kwargs={}, example=False), AIMessage(content='How severe were your symptoms?', additional_kwargs={}, example=False), HumanMessage(content='Mild', additional_kwargs={}, example=False), AIMessage(content='Have you had any previous allergic reactions?', additional_kwargs={}, example=False), HumanMessage(content='Yes', 

In [4]:
ALLERGEN_DIAGNOSE_CONST = \
{"inputs":["problem", "chat_history"],
 "outputs": {"chain": "return 'diagnose'.",
             "allergens": "a js array of allergens.",
             "reason": "a explanation for your diagnose.", 
             "guide": "a helpful guide to avoid the problem and also point out mistake that your patient is making."},
 "template": """You are a food allergist and you have asked your patient some questions about the problem "{problem}"
 {chat_history}

Mission: Based on what your patient have told, please diagose the allergen (or allergens) that causing the problem and explain the reason.

{format_instructions}

Output:"""}

from langchain.llms import BaseLLM
from langchain import LLMChain
import sys
import os
# sys.path.append(f"{os.path.dirname(__file__)}/../..")
sys.path.append('../')
from botcore.utils import build_prompt
from botcore.setup import trace_palm2

def build_allergen_diagnose_chain(model, memory):
    inputs = ALLERGEN_DIAGNOSE_CONST['inputs']
    outputs = ALLERGEN_DIAGNOSE_CONST['outputs']
    template = ALLERGEN_DIAGNOSE_CONST['template']
    prompt = build_prompt(inputs, outputs, template, include_parser=True)
    chain = LLMChain(llm=model, prompt=prompt, output_key='result', verbose=True, memory=memory)
    return chain


In [5]:
from botcore.utils import parse_nested_json
from pprint import pprint as ppt
model = trace_palm2()
chain = build_allergen_diagnose_chain(model, memory.memory)
ans = chain({"problem": p})
data = parse_nested_json(ans['result'])
ppt(data)

Enable tracing at test-deploy
Vertex AI Palm 2 ready


> Entering new  chain...
Prompt after formatting:
You are a food allergist and you have asked your patient some questions about the problem "Eating peanuts made my face swollen."
 Human: Eating peanuts made my face swollen.
AI: What other foods have you eaten recently?
Human: Peanuts
AI: Have you had any other symptoms besides swelling?
Human: Hives
AI: How long did it take for your symptoms to start after eating the peanuts?
Human: Less than 1 hour
AI: How severe were your symptoms?
Human: Mild
AI: Have you had any previous allergic reactions?
Human: Yes
AI: Do you have any family history of allergies?
Human: Yes
Human: Eating peanuts made my face swollen.
AI: ```json
{
	"chain": "diagnose",
	"allergens": ["peanuts"],
	"reason": "Your symptoms are consistent with a peanut allergy. Peanuts are a common allergen, and can cause a variety of symptoms, including swelling, hives, and difficulty breathing. If you have a peanut allergy, i